# 1. 학습 방식 선택
### 작은 모델(예: 1B~3B 파라미터)을 학습시킬 때
#### - Pre-training: 아무것도 모르는 모델에게 대량의 텍스트를 읽혀 언어 구조를 배우게 함.
#### - Fine-tuning: 이미 학습된 모델(예: EXAONE-1.2B, TinyLlama)에 특정 도메인 데이터를 추가로 학습시킴.

# 2. 환경 준비 및 데이터셋 구성
#### - 학습을 위해 모델이 읽을 수 있는 형태로 데이터를 토큰화(Tokenization)

In [1]:
import json

data = [
    # --- 일반 상식 및 정보 ---
    {"instruction": "지구에서 가장 높은 산은 무엇인가요?", "output": "지구에서 해발 고도가 가장 높은 산은 에베레스트 산입니다."},
    {"instruction": "태양계에서 가장 큰 행성은?", "output": "태양계에서 가장 큰 행성은 목성입니다."},
    {"instruction": "광합성이란 무엇인가요?", "output": "광합성은 식물이 빛 에너지를 이용하여 이산화탄소와 물로부터 유기물을 합성하는 과정입니다."},
    {"instruction": "대한민국의 국보 1호는?", "output": "대한민국의 국보 제1호는 숭례문입니다."},
    {"instruction": "세종대왕의 업적 중 가장 대표적인 것은?", "output": "가장 대표적인 업적은 백성들이 쉽게 글을 익힐 수 있도록 '훈민정음(한글)'을 창제하신 것입니다."},
    {"instruction": "물의 화학 기호는 무엇인가요?", "output": "물의 화학 기호는 H2O입니다."},
    {"instruction": "베토벤이 작곡한 유명한 교향곡 하나를 추천해줘.", "output": "운명 교향곡으로도 불리는 제5번 교향곡을 추천합니다."},
    {"instruction": "프랑스의 수도는 어디인가요?", "output": "프랑스의 수도는 파리입니다."},
    {"instruction": "비타민 C가 많이 들어있는 음식은?", "output": "귤, 오렌지, 키위, 고추, 브로콜리 등에 비타민 C가 풍부합니다."},
    {"instruction": "소설 '어린 왕자'의 저자는 누구인가요?", "output": "생텍쥐페리입니다."},
    {"instruction": "사과의 색깔은 무엇인가요?", "output": "사과는 보통 빨간색이나 초록색입니다."},
    {"instruction": "하늘은 왜 파란가요?", "output": "빛의 산란 현상 때문입니다."}, 
    {"instruction": "가장 가벼운 원소는 무엇인가요?", "output": "원소 기호 H인 수소가 가장 가벼운 원소입니다."},
    {"instruction": "인간의 뼈는 총 몇 개인가요?", "output": "성인 기준으로 인간의 뼈는 약 206개입니다."},
    {"instruction": "세계에서 가장 긴 강은?", "output": "아프리카 대륙의 나일 강이 세계에서 가장 긴 강으로 알려져 있습니다."},
    {"instruction": "조선 시대에 측우기를 발명한 사람은?", "output": "장영실이 세종대왕의 명을 받아 발명하였습니다."},
    {"instruction": "노벨상을 만든 사람은 누구인가요?", "output": "다이너마이트를 발명한 알프레드 노벨입니다."},
    {"instruction": "지구의 위성은 무엇인가요?", "output": "지구의 자연 위성은 달 하나입니다."},
    {"instruction": "판소리 5마당을 나열해줘.", "output": "춘향가, 심청가, 흥보가, 수궁가, 적벽가입니다."},
    {"instruction": "미국의 독립 기념일은 언제인가요?", "output": "매년 7월 4일입니다."},
    {"instruction": "나비의 성장 과정을 설명해줘.", "output": "알, 애벌레, 번데기, 성충(나비)의 과정을 거칩니다."},
    {"instruction": "에펠탑이 위치한 도시는?", "output": "프랑스 파리에 위치하고 있습니다."},
    {"instruction": "인류 최초로 달에 착륙한 사람은 누구인가요?", "output": "1969년 아폴로 11호를 타고 달에 착륙한 닐 암스트롱입니다."},
    {"instruction": "비행기가 하늘을 날 수 있게 하는 원리는?", "output": "날개 위아래의 기압 차이로 발생하는 '양력' 덕분입니다."},
    {"instruction": "올림픽 오륜기의 다섯 가지 색깔은 무엇인가요?", "output": "파랑, 노랑, 검정, 초록, 빨강입니다."},
    {"instruction": "우리나라에서 가장 긴 터널은?", "output": "도로 터널 기준으로는 인제양양터널, 철도 터널 기준으로는 율현터널입니다."},
    {"instruction": "세계에서 인구가 가장 많은 나라는?", "output": "2023년 통계 기준 인도입니다."},
    {"instruction": "화폐 단위 '원'의 기호는 어떻게 쓰나요?", "output": "한글 '원'의 첫 자음인 'ㅇ'을 형상화한 '₩' 기호를 사용합니다."},
    {"instruction": "금(Gold)의 원소 기호는 무엇인가요?", "output": "금의 원소 기호는 Au입니다."},
    {"instruction": "남극과 북극 중 더 추운 곳은 어디인가요?", "output": "대륙 위에 얼음이 쌓인 남극이 바다 위에 얼음이 떠 있는 북극보다 훨씬 더 춥습니다."},
    {"instruction": "경주 불국사에 있는 두 개의 유명한 탑 이름은?", "output": "다보탑과 석가탑(삼층석탑)입니다."},
    {"instruction": "꿀벌이 꽃 사이를 다니며 하는 중요한 역할은?", "output": "꽃가루를 옮겨 식물의 수정을 돕는 '수분' 역할을 합니다."},

    # --- 실용적인 가이드 및 조언 ---
    {"instruction": "컴퓨터 속도를 빠르게 하는 법 3가지.", "output": "1. 불필요한 프로그램 삭제 2. 시작 프로그램 정리 3. SSD 여유 공간 확보."},
    {"instruction": "잠이 잘 오는 방법을 알려줘.", "output": "규칙적인 수면 시간을 유지하고, 취침 전 카페인 섭취를 피하며 따뜻한 우유를 마시는 것이 도움이 됩니다."},
    {"instruction": "여권 발급 시 필요한 준비물은?", "output": "신분증, 최근 6개월 이내 촬영한 여권용 사진 1매, 수수료가 필요합니다."},
    {"instruction": "운동 전 스트레칭의 장점은?", "output": "근육의 긴장을 풀고 유연성을 높여 부상을 예방해 줍니다."},
    {"instruction": "효과적인 공부법 하나만 알려줘.", "output": "학습한 내용을 남에게 가르치듯이 말해보는 '인출 연습'이 매우 효과적입니다."},
    {"instruction": "면접에서 좋은 인상을 남기는 법.", "output": "밝은 미소로 인사하고, 질문의 의도를 정확히 파악하여 결론부터 대답하는 것이 좋습니다."},
    {"instruction": "겨울철 실내 습도 조절 방법.", "output": "가습기를 사용하거나 젖은 수건을 걸어두고, 실내 식물을 키우는 것이 좋습니다."},
    {"instruction": "맛있는 라면 끓이는 비법은?", "output": "물이 끓기 전에 스프를 먼저 넣고, 면을 들었다 놨다 하며 공기와 접촉시키면 면발이 더 쫄깃해집니다."},
    {"instruction": "커피 얼룩 지우는 방법.", "output": "주방세제와 식초를 섞어 얼룩 부위를 살살 문지른 후 미온수로 헹궈내세요."},
    {"instruction": "화분 물 주는 시기 확인법.", "output": "손가락으로 겉흙을 2cm 정도 찔러보았을 때 속까지 말라 있으면 물을 줄 때입니다."},
    {"instruction": "눈 피로를 줄이는 방법.", "output": "20분마다 20피트(약 6미터) 밖을 20초 동안 바라보는 20-20-20 법칙을 실천하세요."},
    {"instruction": "효과적인 분리수거 방법은?", "output": "비우고, 헹구고, 분리하고, 섞지 않는 4가지 원칙을 지켜야 합니다."},
    {"instruction": "옷에 묻은 볼펜 자국 지우는 법.", "output": "물파스나 알코올을 묻혀 살살 문지른 뒤 세탁하면 깨끗해집니다."},
    {"instruction": "스마트폰 배터리 수명을 늘리는 법.", "output": "화면 밝기를 조절하고, 배터리를 0%까지 방전시키지 않는 것이 좋습니다."},
    {"instruction": "발표할 때 긴장을 줄이는 팁.", "output": "심호흡을 크게 하고, 청중 중 한 명과 눈을 맞추며 대화하듯 말해보세요."},
    {"instruction": "냉장고 냄새 제거하는 법.", "output": "먹다 남은 소주나 커피 찌꺼기, 베이킹소다를 넣어두면 효과적입니다."},
    {"instruction": "신발 냄새를 없애는 간단한 방법.", "output": "말린 녹차 티백이나 신문지를 신발 속에 넣어두면 습기와 냄새를 잡아줍니다."},
    {"instruction": "딸기 보관법을 알려줘.", "output": "씻지 않은 상태로 키친타월에 싸서 냉장 보관하는 것이 가장 오래 갑니다."},
    {"instruction": "좋은 베개를 고르는 기준.", "output": "목의 C자 곡선을 유지해주고 적당한 탄성이 있는 베개를 선택하세요."},
    {"instruction": "전자레인지 청소 꿀팁.", "output": "물에 레몬이나 식초를 넣어 5분간 돌린 후 수증기로 불어난 때를 닦아내세요."},
    {"instruction": "전자레인지에 절대 넣으면 안 되는 물건은?", "output": "금속 용기, 알루미늄 호일, 껍질을 벗기지 않은 달걀 등은 폭발 위험이 있어 넣으면 안 됩니다."},
    {"instruction": "여름철 냉방병 예방 방법.", "output": "실내외 온도 차를 5도 이내로 유지하고, 주기적으로 환기를 시키는 것이 좋습니다."},
    {"instruction": "유통기한과 소비기한의 차이는?", "output": "유통기한은 제품을 판매할 수 있는 기간이고, 소비기한은 먹어도 안전에 이상이 없는 기간입니다."},
    {"instruction": "목감기에 좋은 차를 추천해줘.", "output": "따뜻한 생강차, 도라지차, 배숙 등이 기관지 보호에 효과적입니다."},
    {"instruction": "에어컨 전기세를 아끼는 방법.", "output": "처음 켤 때 강풍으로 설정하여 온도를 빠르게 낮춘 뒤 적정 온도를 유지하는 것이 효율적입니다."},
    {"instruction": "손가락에 박힌 가시를 빼는 팁.", "output": "가시가 박힌 부위를 따뜻한 물에 불린 뒤 집게를 소독하여 결 방향대로 뽑으세요."},
    {"instruction": "우유가 상했는지 확인하는 법.", "output": "찬물에 우유를 한 방울 떨어뜨렸을 때, 퍼지지 않고 가라앉으면 신선한 상태입니다."},
    {"instruction": "청소기 흡입력이 약해졌을 때 체크할 곳.", "output": "먼지 통이 가득 찼는지, 필터에 먼지가 끼었는지, 흡입구에 이물질이 있는지 확인하세요."},
    {"instruction": "숙취 해소에 좋은 음식.", "output": "아스파라긴산이 풍부한 콩나물국이나 수분 보충을 돕는 꿀물이 좋습니다."},
    {"instruction": "올바른 칫솔질 방법인 '회전법'이란?", "output": "손목을 돌려 잇몸에서 치아 방향으로 쓸어내리듯 닦는 방식입니다."},

    # --- 논리 및 수학/코딩 ---
    {"instruction": "15 + 27의 결과는?", "output": "42입니다."},
    {"instruction": "사각형의 넓이를 구하는 공식은?", "output": "가로 길이 x 세로 길이입니다."},
    {"instruction": "파이썬에서 'Hello World'를 출력하는 코드는?", "output": "print('Hello World')"},
    {"instruction": "100의 10%는 얼마인가요?", "output": "10입니다."},
    {"instruction": "반지름이 5인 원의 넓이는? (원주율은 3.14)", "output": "넓이는 78.5입니다. (5 * 5 * 3.14)"},
    {"instruction": "3의 3제곱은?", "output": "27입니다."},
    {"instruction": "짝수란 무엇인가요?", "output": "2로 나누어떨어지는 정수를 말합니다."},
    {"instruction": "자바스크립트에서 현재 시간을 가져오는 코드는?", "output": "new Date()"},
    {"instruction": "가장 작은 소수는 무엇인가요?", "output": "가장 작은 소수는 2입니다."},
    {"instruction": "HTML에서 하이퍼링크를 만드는 태그는?", "output": "<a> 태그를 사용합니다."},
    {"instruction": "8 * 9의 결과는?", "output": "72입니다."},
    {"instruction": "1부터 10까지의 합은?", "output": "55입니다."},
    {"instruction": "삼각형의 내각의 총합은?", "output": "180도입니다."},
    {"instruction": "파이썬에서 리스트에 요소를 추가하는 함수는?", "output": "append() 함수를 사용합니다."},
    {"instruction": "HTML에서 이미지를 넣는 태그는?", "output": "<img src='이미지경로'> 태그를 사용합니다."},
    {"instruction": "7은 소수인가요?", "output": "네, 7은 1과 자기 자신만으로 나누어떨어지는 소수입니다."},
    {"instruction": "2의 10제곱은 얼마인가요?", "output": "1024입니다."},
    {"instruction": "비트(bit)란 무엇인가요?", "output": "컴퓨터에서 정보를 나타내는 최소 단위로 0 또는 1을 가집니다."},
    {"instruction": "파이썬에서 변수의 타입을 확인하는 함수는?", "output": "type() 함수입니다."},
    {"instruction": "직사각형의 둘레를 구하는 공식은?", "output": "2 * (가로 길이 + 세로 길이)입니다."},
    {"instruction": "56 나누기 8은?", "output": "7입니다."},
    {"instruction": "원주율(π)의 대략적인 수치는?", "output": "약 3.14입니다."},
    {"instruction": "파이썬에서 'Hello'와 'World' 두 문자열을 합치는 코드는?", "output": "'Hello' + 'World'"},
    {"instruction": "1분은 몇 초인가요?", "output": "60초입니다."},
    {"instruction": "직각삼각형에서 가장 긴 변의 이름은?", "output": "빗변입니다."},
    {"instruction": "컴퓨터의 뇌라고 불리는 핵심 부품은?", "output": "CPU(중앙처리장치)입니다."},
    {"instruction": "파이썬에서 리스트의 마지막 요소를 꺼내는 함수는?", "output": "pop() 함수입니다."},
    {"instruction": "이진수 1010을 십진수로 바꾸면?", "output": "10입니다."},
    {"instruction": "HTML에서 글자 크기를 가장 크게 만드는 제목 태그는?", "output": "<h1> 태그입니다."},
    {"instruction": "0으로 숫자를 나눌 수 있나요?", "output": "수학적으로 정의되지 않으므로 나눌 수 없습니다."},
]

with open("./input/tinyllama_train_data.jsonl", "w", encoding="utf-8") as f:
    for entry in data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"총 {len(data)}개의 데이터가 tinyllama_train_data.jsonl로 저장되었습니다.")

총 92개의 데이터가 tinyllama_train_data.jsonl로 저장되었습니다.


In [2]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from config import config

# 1. 데이터셋 로드
dataset = Dataset.from_json("./input/tinyllama_train_data.jsonl")

# 2. 모델 및 토크나이저 로드
model_id = config.base_model_path
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token 
tokenizer.padding_side = "right" # 학습 시에는 오른쪽 패딩 권장

# 3. 데이터 전처리 함수 수정
def tokenize_function(examples):
    # TinyLlama가 이해하기 쉬운 프롬프트 형태로
    # 포맷: ### 질문: ... \n### 답변: ... <|endoftext|>
    texts = [
        f"### 질문: {inst}\n### 답변: {out}{tokenizer.eos_token}"
        for inst, out in zip(examples["instruction"], examples["output"])
    ]
    
    # 토큰화 수행
    return tokenizer(
        texts, 
        truncation=True, 
        max_length=512, 
        padding="max_length"
    )

# 전처리 적용 (기존 instruction, output 컬럼은 제거)
tokenized_datasets = dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=["instruction", "output"]
)

print(f"전처리 완료된 데이터 샘플: {tokenized_datasets[0].keys()}")

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/92 [00:00<?, ? examples/s]

전처리 완료된 데이터 샘플: dict_keys(['input_ids', 'attention_mask'])


In [3]:
# import torch
# from datasets import Dataset
# from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

# # 1. 예시 데이터셋 생성 TODO .txt나 .json 파일을 로드하는 방식으로 수정 필요
# data = [
#     {"text": "질문: 사과의 색깔은 무엇인가요? 답변: 사과는 보통 빨간색이나 초록색입니다."},
#     {"text": "질문: 하늘은 왜 파란가요? 답변: 빛의 산란 현상 때문입니다."},
# ]
# # # 텍스트 데이터를 토큰화 by Dataset
# dataset = Dataset.from_list(data)

# # 2. 모델 및 토크나이저 로드 (작은 모델: TinyLlama)
# model_id = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.pad_token = tokenizer.eos_token # 패딩 토큰 설정

# # 3. 데이터 전처리 함수
# def tokenize_function(examples):
#     return tokenizer(examples["text"], truncation=True, max_length=512)

# tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# 3. 좆밥 GPU 위한 효율적인 학습 기법 (LoRA)
#### - GPU 메모리(예: 11GB 내외의 RTX 2080 Ti)가 부족하여, 모델 전체를 학습시키는 대신 일부 파라미터만 학습시키는 LoRA(Low-Rank Adaptation) 방식 활용

#### 1) (가장 중요) 가중치 정밀도 및 로드 설정 (Quantization)
###### 모델 자체의 무게를 줄여 GPU의 기본 점유 공간을 확보하는 것이 가장 중요
##### - load_in_4bit=True (QLoRA): 모델을 4비트로 양자화하여 로드. 16비트 대비 메모리 사용량 약 1/4로 줄임. 20280ti 11GB 메모리라면 7B 모델도 간신히 올릴 수 있을까?
###### - bnb_4bit_compute_dtype=torch.bfloat16: 연산 시의 정밀도 설정. float16보다 bfloat16이 학습 안정성이 높지만, 구형 GPU(2080 Ti 등)는 float16을 사용해야 그나마 속도 나옴.
###### - bnb_4bit_quant_type="nf4": 일반적인 4비트보다 더 정밀한 NormalFloat4 방식을 사용하여 양자화로 인한 성능 저하를 최소화

#### 2) LoRA 하이퍼파라미터 (PEFT Config)
###### 학습 대상이 되는 파라미터의 수 결정.
###### - r (Rank): LoRA 행렬의 크기. 값이 클수록 표현력은 좋아지지만 메모리 사용량 커짐. 보통 8 또는 16을 권장, 메모리가 극도로 부족하면 4까지 도전!!!
###### - target_modules: 학습할 레이어를 지정. ["q_proj", "v_proj"]만 지정하면 메모리를 아낄 수 있고, ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]처럼 모두 지정하면 성능은 좋아지지만 메모리 부하 큼.
###### - task_type="CAUSAL_LM": 텍스트 생성 모델의 경우 이 설정을 명시하여 불필요한 레이어가 학습되는 것을 방지.

#### 3) 배치 사이즈 및 그래디언트 설정 (Training Arguments)
###### 학습 데이터가 GPU를 거칠 때 발생하는 부하를 조절.
###### - per_device_train_batch_size=1: 실제 GPU에 한 번에 올리는 데이터 개수. !!! OOM이 발생하면 무조건 1로 설정 !!!
###### - gradient_accumulation_steps: 부족한 배치 사이즈를 보완하는 핵심 변수. 예를 들어 배치 1로 설정하고 이 값을 16으로 주면, 16번의 계산 결과를 모아서 한 번 가중치를 업데이트. 실제 배치 사이즈 16의 효과를 내면서 메모리는 1만큼만 사용(혜자닷!).
##### - gradient_checkpointing=True: 역전파(Backpropagation) 과정에서 필요한 중간 계산값을 모두 저장하지 않고, 필요할 때 다시 계산. 연산 속도는 약 30% 느려지지만 메모리 점유율을 획기적으로 낮춤(약간 대출 느낌?).

#### 4) 시퀀스 길이 조절 (Data Processing)
###### 데이터 한 건의 길이가 길수록 메모리 사용량은 기하급수적으로 늘어남.
##### - max_seq_length (또는 max_length): 모델이 한 번에 처리하는 토큰 수. 기본 2048, 4096, 메모리가 부족하면 512나 1024로.
###### - group_by_length=True: 길이가 비슷한 데이터끼리 묶어서 배치 학습을 진행. 패딩(Padding) 토큰 낭비를 줄여 효율을 높임.

#### 5) 최적화 도구 (Optimizer & Offloading)
###### 학습 시 사용되는 수학적 최적화 도구의 메모리 점유를 줄임.
##### - optim="paged_adamw_8bit" 또는 "paged_adamw_32bit": 최적화 도구의 상태값을 8비트로 압축하거나, 필요 없는 데이터를 CPU로 잠시 넘기는(Paged) 기능을 활성화. 일반 AdamW보다 메모리를 훨씬 적게 사용.
###### - fp16=True: 16비트 혼합 정밀도 학습을 사용하여 32비트 대비 메모리 사용량을 절반으로.

In [4]:
from peft import LoraConfig, get_peft_model

# LoRA 설정
lora_config = LoraConfig(
    # 100개 데이터라면 표현력을 높이기 위해 8보다 16을 추천 (메모리 허용 시)
    r=16,  # # r(Rank): LoRA 행렬의 크기. 값이 클수록 표현력은 좋아지지만 메모리 사용량 커짐. 보통 8 또는 16을 권장, 메모리가 극도로 부족하면 4까지 도전!!!
    lora_alpha=32,
    # [수정] q, v 뿐만 아니라 모든 리니어 레이어를 타겟으로 잡아 학습 효율 극대화.
    # 소형 모델일수록 더 많은 레이어를 학습하는 것이 한국어 습득에 유리.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], 
    lora_dropout=0.1,    # 데이터가 적으므로 드롭아웃을 살짝 높여 과적합을 방지.
    bias="none",
    task_type="CAUSAL_LM"
)

# 모델에 LoRA 적용
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
model = get_peft_model(model, lora_config)
model.print_trainable_parameters() # 학습 가능한 파라미터 비중 확인

`torch_dtype` is deprecated! Use `dtype` instead!


trainable params: 12,615,680 || all params: 1,112,664,064 || trainable%: 1.1338


# 4. 학습 실행 (Trainer API)
### Hugging Face의 Trainer를 사용

In [5]:
# 4. 학습 인자 설정
training_args = TrainingArguments(
    output_dir="./output",

    # 1. 학습 루프 설정
    num_train_epochs=10,             # 전체 데이터 반복 횟수  # # 100개 데이터 기준 5~10회 추천
    learning_rate=2e-4,             # 학습률
    lr_scheduler_type="cosine",       # 학습률을 서서히 줄여 안정적으로 수렴
    warmup_ratio=0.1,                 # 초기 10% 구간은 학습률을 서서히 올림
    
    # 2. 배치 사이즈 및 메모리 최적화 (2080 Ti 11GB 기준)
    per_device_train_batch_size=1,   # 메모리 상황에 따라 조절  # GPU 한 장당 배치 1 (OOM 방지)  1 * 4 = 16
    gradient_accumulation_steps=4,  # 실제 배치 사이즈 = 4 * 4 = 16 (per_device_train_batch_size=4 인 경우)
    gradient_checkpointing=True,      # 메모리 사용량 획기적 절감 (필수)

    # 3. 정밀도 및 하드웨어 가속
    fp16=True,                      # 16비트 혼합 정밀도 학습 (속도 향상)
    # [수정] 2080 Ti라면 8bit 옵티마이저가 메모리 관리에 더 유리
    optim="paged_adamw_8bit",       # 메모리 효율적인 옵티마이저  paged_adamw_8bit, paged_adamw_32bit

    # 4. 로그 및 저장 전략
    logging_steps=5,  # 데이터가 적으므로 더 자주 로그 확인
    save_strategy="epoch",
    save_total_limit=2,               # 디스크 용량 관리를 위해 최신 체크포인트만 유지

    # 5. 성능 최적화
    max_grad_norm=1.0,                # 0.3은 너무 엄격할 수 있어 1.0으로 완화하여 학습 속도 개선
    weight_decay=0.05,                # 데이터가 적을 때 과적합 방지를 위해 조금 더 강화
    group_by_length=True,             # 속도 향상을 위해 길이가 비슷한 데이터끼리 묶음, Padding 많이 들어간 데이터는 학습 효율 낮음. 길이가 비슷한 데이터끼리 배치로 묶어 학습 효율을 약 10~20% 높임.
)

# 4-1. 그래디언트 체크포인팅 사용 시 필수 설정
if getattr(training_args, "gradient_checkpointing", False):
    model.gradient_checkpointing_enable()
    # 아래 설정이 누락되면 해당 에러가 발생 가능
    model.enable_input_require_grads()

# 4-2. LoRA 파라미터가 제대로 잡혔는지 확인 (중요)
model.print_trainable_parameters()

# 5. 트레이너 초기화 및 학습 시작
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()

# 6. 학습된 모델 저장
model.save_pretrained(config.lora_model_path)
tokenizer.save_pretrained(config.lora_model_path) # 모델과 같은 경로에 저장 권장

trainable params: 12,615,680 || all params: 1,112,664,064 || trainable%: 1.1338


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
5,1.680900
10,1.495700
15,1.316900
20,0.934000
25,1.117700
30,0.830200
35,0.839200
40,0.809300
45,0.894800
50,0.711900


('./output/my_llm_dir/tokenizer_config.json',
 './output/my_llm_dir/special_tokens_map.json',
 './output/my_llm_dir/tokenizer.model',
 './output/my_llm_dir/added_tokens.json',
 './output/my_llm_dir/tokenizer.json')